# Requirements

In [0]:
!pip install larq tensorflow-datasets

In [0]:
import tensorflow as tf
from tensorflow import keras
import larq as lq
import numpy as np
import tensorflow_datasets as tfds
from tensorflow.keras.layers import *

> # Get Oxford pet dataset dataset from tf datasets

In [0]:
train_data, info_1 = tfds.load(name="oxford_iiit_pet:3.0.0", split="train",with_info=True) 
test_data, info_2 = tfds.load(name="oxford_iiit_pet:3.0.0", split="test",with_info=True)


In [0]:
def normalize_data(data_val):
    
    image = data_val['image']
    mask = data_val['segmentation_mask']
    image = tf.image.resize(image, (128, 128))
    mask = tf.image.resize(mask, (128, 128))
    image = tf.cast(image, tf.float32)
    # Normalize the pixel values
    image = image / 255.0
    # Change mask vals to [0,1,2]
    mask -= 1
    return image, mask

In [0]:
def data_augmentation(data_val):
    
    #Normalize both training and testing images
   
    image, mask = normalize_data(data_val)
    #Augmentation only for training
    if tf.random.uniform(()) > 0.5:
        #Apply to both image and maska
        image = tf.image.flip_left_right(image)
        mask = tf.image.flip_left_right(mask)
  
    
    return image, mask

In [0]:
print("Training samples: {} \nTest samples: {} ".format(info_1.splits['train'].num_examples,info_2.splits['test'].num_examples))

In [0]:
train = train_data.map(data_augmentation, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test = test_data.map(normalize_data)


In [0]:
train_samples = info_1.splits['train'].num_examples
batch_size = 64

In [0]:
train_dataset = train.cache().shuffle(1000).batch(batch_size).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test.batch(batch_size)

In [0]:
def conv_block(n_filters,X,kwargs,reduce = True):
    
   
    Y = lq.layers.QuantConv2D(n_filters,3, padding='same', **kwargs)(X)
    Y = BatchNormalization(scale=False)(Y)
    Y = lq.layers.QuantConv2D(n_filters,3, padding='same', **kwargs)(Y)
    Y = BatchNormalization(scale=False)(Y)
    if reduce:
        Y = MaxPool2D(pool_size=(2,2),strides=(2,2))(Y)
    
    return Y



In [0]:
# All quantized layers except the first will use the same options

def create_model(kwargs):
    
    inputs = tf.keras.Input(shape=(128,128,3))
    
    conv0 = lq.layers.QuantConv2D(32, 3, padding='same',
                              kernel_quantizer="ste_sign",
                              kernel_constraint="weight_clip",
                              use_bias=False)(inputs)
    
    conv0 = BatchNormalization(scale=False)(conv0)
    
    conv1 = conv_block(64,conv0,kwargs)
    
    conv2 = conv_block(128,conv1,kwargs)
    
    conv3 = conv_block(256,conv2,kwargs)
    
    conv4 = conv_block(512,conv3,kwargs)
    
    up1 = concatenate([UpSampling2D()(conv4),conv3],axis=3)
    up1 = conv_block(256,up1,kwargs,reduce=False)
    
    up2 = concatenate([UpSampling2D()(up1),conv2],axis=3)
    up2 = conv_block(128,up2,kwargs,reduce=False)
    
    up3 = concatenate([UpSampling2D()(up2),conv1],axis=3)
    up3 = conv_block(64,up3,kwargs,reduce=False)
    
    up4 = concatenate([UpSampling2D()(up3),conv0],axis=3)
    up4 = conv_block(32,up4,kwargs,reduce=False)
    
    l_out = lq.layers.QuantConv2D(3,1,padding='same',**kwargs)(up4)
    outputs = Activation("softmax")(l_out)
    
    model = tf.keras.Model(inputs=[inputs],outputs=[outputs])
    
    return model
   

In [0]:
model_params = dict(input_quantizer="ste_sign",
              kernel_quantizer= tf.keras.layers.Activation("linear"),
              kernel_constraint= None,
              use_bias=False)

In [0]:
#Return suitable optimizer bop or adam
def optimizer_fn(bop=True):
    if bop:
        initial_lr = 1e-2
    else:
        initial_lr = 1e-3
    threshold_val = 1e-8
    gamma_val = 1e-4
    gamma_decay = 0.1
    #Decay after 100 epochs for batches of 64
    decay_step = int((info_1.splits['train'].num_examples / batch_size) * 100)
    adam = tf.keras.optimizers.Adam(lr=initial_lr)
    
    if bop: 
        optimizer=lq.optimizers.Bop(fp_optimizer=adam, threshold= threshold_val, gamma=tf.keras.optimizers.schedules.ExponentialDecay(
            gamma_val, decay_step, gamma_decay, staircase=True))
        return optimizer
    
    else:
        return adam
   


In [0]:
def lr_schedule(epoch):
    return 1e-2 * 0.1 ** (epoch // 100)

In [0]:
def train_model(bop=True):
    model = create_model(model_params)
    model.compile(
        optimizer=optimizer_fn(bop),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    logs = keras.callbacks.CSVLogger('seg_bop.log')
    callbacks = [logs]
    if not bop:
        scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)
        callbacks.append(scheduler)
    
    trained_model = model.fit(
    train_dataset,
    epochs=300,
    steps_per_epoch= info_1.splits['train'].num_examples // batch_size,
    validation_data= test_dataset,
    validation_steps= info_1.splits['test'].num_examples // batch_size,
    verbose=1,
    callbacks=callbacks
    )
    if bop:
        model.save('seg_bop.h5')
    else:
        model.save('seg_baseline.h5')

    

In [0]:
# For baseline set bop to false
train_model(bop=True)
